### Giovanni Gamaliel López Padilla
### Procesamiento de lenguaje natural
### Tarea 04

### README

#### Organización de la carpeta

```bash
├── Data
│  ├── conferences
│  │  ├── 2018-12-07
│  │  ├── 2018-12-10
│  │  ├── .........
│  │  ├── 2021-01-21
│  │  └── 2021-01-22
│  ├── mex_train.txt
│  ├── mex_train_labels.txt
│  ├── mex_val.txt
│  └── mex_val_labels.txt
├── Modules
│  ├── datasets.py
│  ├── dictionary.py
│  ├── functions.py
│  ├── mañaneras.py
│  ├── models.py
│  ├── tweets.py
│  └── vocabulary.py
└── Tarea04.ipynb
```

Las funciones y clases contenidad en los archivos de la carpeta ``Modules`` se encuentran contenidas en el notebook.

Si se quiere cambiar la posición de los archivos de datos debera modificarse la función ``obtain_parameters`` en los apartados correspondientes a cada base de datos.

In [30]:
from sklearn.model_selection import train_test_split
from nltk import bigrams as bigrams_nltk
from nltk.tokenize import TweetTokenizer
from numpy import array, log, zeros, exp
from nltk import FreqDist,ngrams
from unidecode import unidecode
from tabulate import tabulate
from numpy.linalg import norm
from random import choice
from os import listdir
import re

In [31]:
def obtain_parameters() -> dict:
    """
    Obtiene las rutas y nombres de los archivos que seran usados
    """
    parameters = {
        # Ruta de los archivos
        "path data": "Data/",
        "path graphics": "Graphics/",
        "path results": "Results/",
        "path conferences": "Data/conferences/",
        # Archivos de entrenamiento
        "train": {
            "data": "mex_train.txt",
            "labels": "mex_train_labels.txt"
        },
        # Archivos de validacion
        "validation": {
            "data": "mex_val.txt",
            "labels": "mex_val_labels.txt"
        },
        "max words": 5000,
        "max bigrams": 1000,
        "lambda list": [[1/3, 1/3, 1/3],
                        [0.4, 0.4, 0.2],
                        [0.2, 0.4, 0.4],
                        [0.5, 0.4, 0.1],
                        [0.1, 0.4, 0.5]]
    }
    return parameters

In [32]:
def tokenize(text: str) -> list:
    """
    Realiza la tokenizacion de un texto dado
    ------------
    Inputs:
    text -> string de texto

    ------------
    Outputs:
    aux -> lista de tokens
    """
    # Expresión para obtener palabras
    reg_exp = r"<?/?[A-Z|a-z]+>?"
    tokenizer = TweetTokenizer().tokenize
    text = unidecode(text)
    # coniverto texto a minúsculas y limpio
    aux = re.findall(reg_exp, text.lower())
    aux = ' '.join(aux)
    # tokenizo
    aux = tokenizer(aux)
    return aux


def ls(path: str) -> list:
    return sorted(listdir(path))


def join_path(path: str, filename: str) -> str:
    """
    Une la direccion de un archivo con su nombre
    """
    return "{}{}".format(path, filename)


def mask_unknow(tweet: str, vocabulary: list) -> str:
    """
    Enmascaramiento de una oración dado un vocabulario
    -----------
    Inputs:
    + tweet -> string con el tweet a enmascarar
    + vocabulary -> vocabulario de los datos de entrenamiento

    ------------
    Outputs:
    tweet_mask -> string con el tweet enmascarado
    """
    # Tokens del tweet dado
    tokens = tokenize(tweet)
    # Enmascaramiento de los tokens
    tweet_mask = [word if word in vocabulary else "<unk>"
                  for word in tokens]
    # Union de los tokens
    tweet_mask = " ".join(tweet_mask)
    return tweet_mask


class dictionary_class:
    """
    Métodos para crear diccionarios con diferentes informaciones
    """

    def __init__(self) -> None:
        pass

    def build_word_index(self, vocabulary: list) -> dict:
        """
        Crea un diccionario con la posición de mayor a menor frecuencia de cada palabra. La llave es la palabra a consultar
        """
        # Inicializacion del diccionario
        index = dict()
        # Inicializacion de la posicion
        for i, word in enumerate(vocabulary):
            index[word] = i
        return index

    def build_with_words_and_documents(self, words: dict, data: list) -> dict:
        """
        Crea un diccionario el cual contendra de forma ordenada el indice de cada palabra y su numero de frecuencias en una coleccion
        """
        freq_word_per_document = dict()
        word_count = dict()
        for i, tweet in enumerate(data):
            word_count[i] = 0
        for word in words:
            freq_word_per_document[word] = word_count
        return freq_word_per_document

    def obtain_index_word(self, index_word: dict) -> dict:
        """
        Invierte los valores de un diccionario dado. Los values pasan a ser keys y viceversa
        """
        invert_index = {}
        for word in index_word:
            invert_index[index_word[word]] = word
        return invert_index

    def sort_dict(self, data: dict, reverse: bool = True) -> dict:
        """
        Ordena un diccionario
        """
        aux = sorted(data.items(),
                     key=lambda item: item[1], reverse=reverse)
        dict_sort = {}
        for word, value in aux:
            dict_sort[word] = value
        return dict_sort


class vocabulary_class:
    def __init__(self) -> None:
        pass

    def obtain(self, data: list, max_words: int) -> dict:
        """
        Obtiene la lista de una distribucion de frecuencias de palabras ordenada de mayor a menor a partir de una lista de oraciones
        """
        # Inicializacion de la lista que guardara los tokens
        corpus = []
        for tweet in data:
            # Creacion y guardado de los tokens
            corpus += tokenize(tweet)
        # Creacion de la distribucion de frecuencias
        vocabulary = FreqDist(corpus)
        vocabulary = self.sort_freqdist(vocabulary)
        # print(vocabulary)
        vocabulary = self.split_data(vocabulary, max_words)
        return vocabulary

    def obtain_with_bigrams(self, data: list, max_bigrams: int) -> list:
        """
        Obtiene la lista de una distribucion de frecuencias de palabras ordenada de mayor a menor a partir de una lista de oraciones
        """
        # Inicializacion de la lista que guardara los tokens
        corpus_bigrams = []
        for tweet in data:
            # Creacion y guardado de los tokens
            corpus_bigrams += bigrams_nltk(tokenize(tweet))
        # Creacion de la distribucion de frecuencias
        vocabulary = FreqDist(corpus_bigrams)
        vocabulary = self.sort_freqdist(vocabulary)
        vocabulary = self.split_data(vocabulary, max_bigrams)
        return vocabulary

    def sort_freqdist(self, vocabulary: FreqDist) -> list:
        """
        Ordena la lista de distribucion de frecuencias de palabras de mayor frecuencia a menor
        """
        aux = {}
        for word in vocabulary:
            aux[word] = vocabulary[word]
        aux = dictionary_class().sort_dict(aux)
        return aux

    def split_data(self, data: dict, max_words: int) -> list:
        """
        Realiza la separacion de elementos en una lista dado el numero de elementos que se quieren conservar
        """
        aux = {}
        for i, word in enumerate(data):
            if i >= max_words:
                break
            aux[word] = data[word]
        return aux


class probability_model_class:
    """
    Contenido para enmascarar palabras que no estan contenidas en un vocabulario y calculo de las probabilidades de unigramas, bigramas y trigramas
    """

    def __init__(self) -> None:
        pass

    def obtain_unigram_probabilities_Laplace(self, data: list) -> dict:
        """
        Calcula las probabilidades de cada unigrama en los datos dado
        --------------
        Inputs:
        + data -> lista de strings con cada tweet

        Output:
        + probability -> diccionario con las probabilidades de cada palabra en el vocabulario
        """
        # Tokens de cada oracion
        tokens = []
        for tweet in data:
            # Token de cada tweet
            tokens += tokenize(tweet)
        # Numero de palabras en el documento
        self.words_len = len(tokens)
        # Frecuencias de las palabras
        self.unigram_fdist = FreqDist(tokens)
        # Tamaño del vocabulario
        n_vocabulary = len(self.unigram_fdist)
        # Probabilidades de unigramas
        probability = {
            word: (count + 1.0) / (self.words_len + n_vocabulary)
            for word, count in self.unigram_fdist.items()
        }
        self.unigram_probability = probability.copy()
        return probability

    def obtain_ngrams_probabilities_Laplace(self, data: list, ngram: int) -> dict:
        """
        Calcula las probabilidades de cada ngrama en los datos dado
        --------------
        Inputs:
        + data -> lista de strings con cada tweet
        + ngram -> entero que indica el número de ngramas a calcular

        Output:
        + probability -> diccionario con las probabilidades de cada bigrama en el vocabulario
        """
        # Creacion de la lista de tokens
        tokens = []
        for tweet in data:
            # Tokens de cada tweet
            tokens += tokenize(tweet)
        self.tokens = tokens.copy()
        # ngramas en el documento
        n_grams = ngrams(tokens, n=ngram)
        # frecuencia de cada ngrama
        ngram_fdist = FreqDist(n_grams)
        # numero total de ngramas
        ngram_len = len(ngram_fdist)
        # inicializacion de la probabilidades
        probabilities = {}
        # Apartado para bigrama
        if ngram == 2:
            for (word_i, word_j), count in ngram_fdist.items():
                # Frecuencia de la palabra anterior en el bigrama
                count_word_i = self.unigram_fdist[word_i]
                # Probabilidad del bigrama
                probabilities[(word_j, word_i)] = (
                    count + 1.0) / (count_word_i + ngram_len)
            self.bigram_probability = probabilities.copy()
        # Apartado para trigrama
        if ngram == 3:
            # bigramas en el documento
            bigrams = ngrams(tokens, n=2)
            # frecuencias de cada bograma
            self.bigram_fdist = FreqDist(bigrams)
            for (word_i, word_j, word_k), count in ngram_fdist.items():
                # frecuencia del bigrama
                count_fs = self.bigram_fdist[(word_i, word_j)]
                # Suavizado con laplace
                probabilities[(word_k, word_i, word_j)] = (
                    count + 1.0) / (count_fs + ngram_len)
            self.trigram_probability = probabilities.copy()
        return probabilities

    def obtain_unigram_probability(self, word: str) -> float:
        """
        Obtiene la probabilidad que tiene un tweet siguiendo una probabilidad de unigramas
        --------------
        Input:
        + tweet -> string del tweet
        """
        if not word in self.unigram_probability:
            word = "<unk>"
        probability = self.unigram_probability[word]
        return probability

    def compute_ngram_probability(self,
                                  tweet: str,
                                  vocabulary: list,
                                  ngram: int = 2) -> float:
        """
        Calculo de la probabilidad de la oracion dada
        ----------
        Inputs:
        + tweet -> string tweet a calcular su probabilidad
        + vocabulary -> vocabulario de los datos de entrenamiento
        + ngram -> entero que indica la cantidad de ngramas a tomar

        -----------
        Output:
        probability -> probabilidad del tweet dado
        """
        # enmascaro oracion
        tweet = mask_unknow(tweet, vocabulary)
        # Concateno inicio y fin de oración
        s_init = "<s>"
        s_fin = "</s>"
        tweet = "{}{}{}".format(s_init, tweet, s_fin)
        tweet = tokenize(tweet)
        # Tamaño del vocabulario
        vocabulary_len = len(vocabulary)
        probability = 1
        # Apartado para bigrama
        if ngram == 2:
            # Frecuencia de los unigramas
            for i in range(len(tweet) - 1):
                word_i = tweet[i]
                word_j = tweet[i + 1]
                if (word_j, word_i) in self.bigram_probability:
                    probability *= self.bigram_probability[(word_j, word_i)]
                # Si la palabra no se encuentra se da una probabilidad
                else:
                    freq_word_i = self.unigram_fdist[word_i]
                    probability *= 1.0 / (freq_word_i + vocabulary_len)
        # Apartado para trigramas
        if ngram == 3:
            # Bigramas en la oracion
            bigrams = ngrams(self.tokens, n=2)
            # Freceucnia de bigramas
            self.bigrams_fdist = FreqDist(bigrams)
            # Tamaño e bigramas
            bigram_len = len(self.bigrams_fdist)
            for i in range(len(tweet) - 2):
                word_i = tweet[i]
                word_j = tweet[i + 1]
                word_k = tweet[i + 2]
                if (word_k, word_i, word_j) in self.trigram_probability:
                    probability *= self.trigram_probability[(
                        word_k, word_i, word_j)]
                # Si no se encuentra se da una probabilidad
                else:
                    count_fs = 0
                    if (word_i, word_j) in self.bigrams_fdist:
                        count_fs = self.bigrams_fdist[(word_i, word_j)]
                    conditional_prop = 1 / (count_fs + bigram_len)
                    probability *= conditional_prop
        return probability

    def obtain_ngram_probability(self, ngram: list) -> float:
        """
        Obtiene la probabilidad de un ngrama dado
        -----------
        Inputs:
        + ngram -> lista de ngramas

        -----------
        Output:
        + probability -> probabilidad del ngrama dado
        """
        probability = 1.0
        ngram_len = len(ngram)
        # Caso para bigrama
        if ngram_len == 2:
            for i in range(ngram_len - 1):
                word_i = ngram[i]
                word_j = ngram[i + 1]
                if (word_j, word_i) in self.bigram_probability:
                    probability *= self.bigram_probability[(word_j, word_i)]
                # Si la probabilidad no existe se da con al menos un conteo
                else:
                    freq_word_i = self.unigram_fdist[word_i]
                    probability *= 1.0 / (freq_word_i + self.words_len)
        # Caso para trigramas
        if ngram_len == 3:
            for i in range(ngram_len - 2):
                word_i = ngram[i]
                word_j = ngram[i + 1]
                word_k = ngram[i + 2]
                if (word_k, word_i, word_j) in self.trigram_probability:
                    probability *= self.trigram_probability[(
                        word_k, word_i, word_j)]
                # Si la probabilidad no existe se da con al menos un conteo
                else:
                    count_fs = 0.0
                    if (word_i, word_j) in self.bigram_fdist:
                        count_fs = self.bigram_fdist[(word_i, word_j)]
                    conditional_prop = 1.0 / (count_fs + self.words_len)
                    probability *= conditional_prop
        return probability


class language_model_class:

    def __init__(self, data_tr: list, data_test: list, data_val: list, vocabulary: dict):
        self.probability_model = probability_model_class()
        self.data_tr = data_tr
        self.data_test = data_test
        self.data_val = data_val
        self.vocabulary = vocabulary.keys()
        self.obtain_probabilities()

    def obtain_probabilities(self) -> None:
        self.unigram_probability = self.probability_model.obtain_unigram_probabilities_Laplace(
            self.data_tr)
        self.bigram_probability = self.probability_model.obtain_ngrams_probabilities_Laplace(
            self.data_tr,
            ngram=2,
        )
        self.tigram_probability = self.probability_model.obtain_ngrams_probabilities_Laplace(
            self.data_tr,
            ngram=3,
        )
        # Preparo modelo para evaluación
        tokens = []
        for tweet in self.data_tr:
            tokens += tokenize(tweet)
        # Total de palabras
        self.words_len = len(tokens)
        bigrams = ngrams(tokens, n=2)
        # Frecuencias bigramas
        self.bigram_fdist = FreqDist(bigrams)
        # Frecuencias unigramas
        self.unigram_fdist = FreqDist(tokens)

    def compute_perplexity(self, lambda_values: list, use_data_test: bool = True) -> float:
        tokens = []
        if use_data_test:
            data = self.data_test
        else:
            data = self.data_val
        for tweet in data:
            tokens += tokenize(tweet)
        trigrams = ngrams(tokens, n=3)
        perplexity = 0.0
        for (word_i, word_j, word_k) in trigrams:
            aux = 0.0
            aux += lambda_values[0] * self.probability_model.obtain_ngram_probability(
                (word_i, word_j, word_k),)
            aux += lambda_values[1] * self.probability_model.obtain_ngram_probability(
                (word_i, word_j),)
            aux += lambda_values[2] * self.probability_model.obtain_unigram_probability(
                word_i)
            perplexity += log(aux)
        perplexity = -perplexity / self.words_len
        return perplexity

    def tweet_probability(self, tweet: str, lambda_values: list, add_s_tokens: bool = True) -> float:
        """
        Calcula la probabilidad de un tweet por medio de la interpolacion
        ----------------
        Inputs:
        + tweet -> string del tweet a calcular su probabilidad
        + lambda_values -> lista de lambdas para los pesos de cada ngrama

        ----------------
        Outputs:
        + probability -> probabilidad del tweet dado
        """
        # Enmascaro palabras desconocidas
        if add_s_tokens:
            tweet = mask_unknow(tweet, self.vocabulary)
            tweet = "<s>{}</>".format(tweet)
        tokens = tokenize(tweet)
        trigrams = ngrams(tokens, n=3)
        probability = 1.0
        for (word_i, word_j, word_k) in trigrams:
            aux = 0
            # Compruebo valores de lambda
            if lambda_values[0] != 0:
                aux += lambda_values[0] * self.probability_model.obtain_ngram_probability(
                    (word_i, word_j, word_k),)
            if lambda_values[1] != 0:
                aux += lambda_values[1] * self.probability_model.obtain_ngram_probability(
                    (word_i, word_j),)
            if lambda_values[2] != 0:
                aux += lambda_values[2] * self.probability_model.obtain_unigram_probability(
                    word_i)
            probability *= aux
        return probability

    def apply_expectation_maximization(self, lambda_test: list = [], iterations: int = 5) -> array:
        results = []
        ngrams = 3
        if not len(lambda_test):
            lambda_test = [1/ngrams for i in range(ngrams)]
        perplexity = exp(self.compute_perplexity(lambda_test))
        results += [["Inicio",
                     lambda_test.copy(),
                     sum(lambda_test),
                     perplexity]]
        data_len = len(self.data_val)
        # Vectores de distribuciones q_m
        dist = zeros((data_len, ngrams), dtype=float)
        for iteration in range(iterations):
            # Ciclo sobre tokens de validación
            for i, tweet in enumerate(self.data_val):
                dist[i, 0] = self.tweet_probability(tweet,
                                                    [lambda_test[0], 0, 0],
                                                    add_s_tokens=False)
                dist[i, 1] = self.tweet_probability(tweet,
                                                    [0, lambda_test[1], 0],
                                                    add_s_tokens=False)
                dist[i, 2] = self.tweet_probability(tweet,
                                                    [0, 0, lambda_test[2]],
                                                    add_s_tokens=False)
                # Normalizo vector
                dist[i] = dist[i] / norm(dist[i])
            # Update lambdas
            for i in range(ngrams):
                lambda_test[i] = sum(dist[:, i]) / data_len
            perplexity = exp(self.compute_perplexity(lambda_test))
            results += [["Iteración {}".format(iteration+1),
                         lambda_test.copy(),
                         sum(lambda_test),
                         perplexity]]

        print(tabulate(results,
                       headers=["Iteracion", "lambdas", "Suma", "Perplexidad"]))
        return lambda_test


class tweetear_model:
    def __init__(self, language_model: language_model_class, lambdas: list):
        self.language_model = language_model
        self.lambdas = lambdas

    def autocomplete(self, init_text: list):
        # Creo todas las posibles oraciones
        tweets = []
        for word in self.language_model.vocabulary:
            value = "{} {}".format(init_text, word)
            tweets += [value]
        probabilities = []
        for i, tweet in enumerate(tweets):
            probabilities += [[self.language_model.tweet_probability(tweet,
                                                                     self.lambdas), i]]
        # Ordeno oraciones por probabilidad
        probabilities.sort(reverse=True)
        tweet = tweets[probabilities[0][1]]
        return tweet

    def write(self, init_text: list, max_words: int = 50) -> str:
        text = tokenize(init_text)
        result = text.copy()
        for i in range(max_words):
            tweet = self.autocomplete(" ".join(text))
            words = tokenize(tweet)
            word = words[-1]
            text.pop(0)
            text.append(word)
            result += [word]
            if word == '</s>':
                break
        result = " ".join(result)
        return result


class tweets_data:
    def __init__(self, parameters: dict) -> None:
        self.vocabulary_model = vocabulary_class()
        self.dictionary = dictionary_class()
        self.parameters = parameters
        self.read()
        self.initialize()

    def initialize(self):
        self.add_s_tokens()
        self.obtain_vocabulary(use_mask=True)
        self.obtain_word_index()
        self.obtain_index_word()
        self.data_tr_mask = self.mask_tweets(self.data_tr_mask)
        self.data_val_mask = self.mask_tweets(self.data_val_mask)
        self.obtain_vocabulary(use_mask=True)
        self.obtain_word_index()
        self.obtain_index_word()
        self.obtain_data_test()
        self.language_model = language_model_class(self.data_tr_mask,
                                                   self.data_test_mask,
                                                   self.data_val_mask,
                                                   self.vocabulary)

    def get_texts_from_file(self, path_data: str, path_labels: str) -> tuple:
        """
        Obtiene una lista de oraciones a partir de un texto con sus respectivas etiquetas
        """
        # Inicilizacion de las listas
        text = []
        labels = []
        # Apertura de los archivos
        with open(path_data, "r") as f_data, open(path_labels, "r") as f_labels:
            # Recoleccion de las oraciones
            for tweet in f_data:
                text += [tweet]
            # Recoleccion de las etiquedas
            for label in f_labels:
                labels += [label]
        # Etiquedas a enteros
        labels = list(map(int, labels))
        return text, labels

    def read(self) -> None:
        """
        Lectura de los datos de entrenamiento y validacion
        """
        # Definicion de las rutas de cada archivo de datos y validacion
        path_data_tr = join_path(
            self.parameters["path data"],
            self.parameters["train"]["data"],
        )
        path_label_tr = join_path(
            self.parameters["path data"],
            self.parameters["train"]["labels"],
        )
        path_data_val = join_path(
            self.parameters["path data"],
            self.parameters["validation"]["data"],
        )
        path_label_val = join_path(
            self.parameters["path data"],
            self.parameters["validation"]["labels"],
        )
        # Lectura de las oraciones y etiquetas de los datos de entrenamiento
        self.data_tr, self.labels_tr = self.get_texts_from_file(
            path_data_tr,
            path_label_tr,
        )
        # Lectura de las oraciones y etiquetas de los datos de validación
        self.data_val, self.labels_val = self.get_texts_from_file(
            path_data_val,
            path_label_val,
        )

    def add_s_tokens(self) -> None:
        self.data_tr_mask = ["<s>{}</s>".format(tweet)
                             for tweet in self.data_tr]
        self.data_val_mask = ["<s>{}</s>".format(tweet)
                              for tweet in self.data_val]

    def obtain_vocabulary(self, use_mask: bool) -> None:
        if use_mask:
            data = self.data_tr_mask
        else:
            data = self.data_tr
        self.vocabulary = self.vocabulary_model.obtain(data,
                                                       self.parameters["max words"])

    def obtain_word_index(self) -> None:
        self.word_index = self.dictionary.build_word_index(self.vocabulary)

    def obtain_index_word(self) -> None:
        self.index_word = self.dictionary.obtain_index_word(self.word_index)

    def mask_tweets(self, tweets: list) -> None:
        tweets_mask = []
        for tweet in tweets:
            tweet_mask = mask_unknow(tweet,
                                     self.vocabulary.keys())
            tweets_mask += [tweet_mask]
        return tweets_mask

    def obtain_data_test(self) -> None:
        self.data_tr_mask, self.data_test_mask = train_test_split(self.data_tr_mask,
                                                                  train_size=0.89,
                                                                  test_size=0.11,
                                                                  random_state=12345)

    def obtain_perplexity(self, use_data_test: bool) -> None:
        results = []
        for lambda_i in self.parameters["lambda list"]:
            perplexity = self.language_model.compute_perplexity(lambda_i,
                                                                use_data_test=use_data_test)
            results += [[lambda_i, exp(perplexity)]]
        print(tabulate(results, headers=["Lambda", "Perplexity"]))


class AMLO_conferences_model(tweets_data):
    def __init__(self, parameters: dict) -> None:
        self.vocabulary_model = vocabulary_class()
        self.dictionary = dictionary_class()
        self.parameters = parameters
        self.read()
        self.obtain_data_val()
        self.initialize()

    def read(self) -> list:
        """
        Realiza la lectura de todas las conferencias y las reune en un solo string
        Input:
            String: path -> Direccion donde se encuentran todos los archivos

        Output:
            String: Texto plano
        """
        files = ls(self.parameters["path conferences"])
        self.data = []
        for file in files:
            # Direccion y nombre del archivo
            filename = join_path(self.parameters["path conferences"],
                                 file)
            # Apertura del archivo
            file_data = open(filename, "r", encoding="utf-8")
            file_text = file_data.read()
            file_text = file_text.lower()
            file_text = unidecode(file_text)
            file_text = tokenize(file_text)
            file_text = " ".join(file_text)
            # Concadenacion del texto
            self.data += [file_text]
            # Cierre del texto
            file_data.close()

    def obtain_data_val(self) -> None:
        self.data_tr, self.data_val = train_test_split(self.data,
                                                       train_size=0.9,
                                                       test_size=0.1,
                                                       random_state=12345)

#### 2) Modelos de lenguaje y evaluación
##### Punto 1
Preprocese todos los tuits de agresividad (positivos y negativos) según su intu- ición para construir un buen corpus para un modelo de lenguaje (e.g., solo palabras en minúscula, etc.). Agregue tokens especiales de ``<s>`` y ``</s>`` según usted considere (e.g., al inicio y final de cada tuit). Defina su vocabulario y enmascare con ``<unk>`` toda palabra que no esté en su vocabulario.

In [1]:
from Modules.datasets import obtain_parameters
from Modules.tweets import tweets_data

In [2]:
parameters = obtain_parameters()
tweets = tweets_data(parameters)

#### Punto 2

Entrene tres modelos de lenguaje sobre todos los tuits: $𝑃_{𝑢𝑛𝑖𝑔𝑟𝑎𝑚𝑎𝑠}(𝑤_1^𝑛)$, $𝑃_{𝑏𝑖𝑔𝑟𝑎𝑚𝑎𝑠}(𝑤_1^𝑛)$, $𝑃_{𝑡𝑟𝑖𝑔𝑟𝑎𝑚𝑎𝑠}(𝑤_1^𝑛)$. Para cada uno proporcione una interfaz (función) sencilla para $𝑃_{𝑛−𝑔𝑟𝑎𝑚𝑎}(𝑤_1^𝑛)$ y $𝑃_{𝑛−𝑔𝑟𝑎𝑚𝑎}(𝑤_1^𝑛|𝑤_{𝑛−𝑁 +1}^{n-1})$. Los modelos deben tener una estrategia común para lidiar consecuencias no vistas. Puede optar por un suavizamiento Laplace o un Good-Turing discounting. Muestre un par de ejemplos de como funciona, al menos uno con una palabra fuera del vocabulario.

In [3]:
language_model = tweets.language_model
probability = language_model.probability_model

##### Unigrama

In [4]:
probability.obtain_unigram_probability("bien")

0.0023423631608247637

In [5]:
probability.obtain_unigram_probability("hola")

0.00013655031879247502

##### Bigrama

In [6]:
probability.obtain_ngram_probability(("bueno", "esto"))

1.1070641765103124e-05

In [7]:
probability.obtain_ngram_probability(("hola", "como"))

1.1075668970405812e-05

##### Trigrama

In [8]:
probability.obtain_ngram_probability(("como", "estas", "plataformas"))

1.1077018509697929e-05

In [9]:
probability.obtain_ngram_probability(("hola", "como", "estas"))

1.1077141211396162e-05

#### Punto 3

Construya un modelo interpolado con valores $\lambda$ fijos:

$$
 \hat{P}(w_𝑛 |𝑤_{𝑛−2}𝑤_{𝑛−1}) = \lambda_1 𝑃(𝑤_𝑛 |𝑤_{𝑛−2}𝑤_{𝑛−1}) + \lambda_2 𝑃(𝑤_𝑛 |𝑤_{𝑛−2}𝑤_{𝑛−1}) + \lambda_3 𝑃(𝑤_𝑛) \nonumber
$$ 

Para ello experimente con el modelo en particiones estratificadas de 80%, 10% y 10% para entrenar (train), ajuste de parámetros (val) y prueba (test) respectivamente. Muestre como bajan o suben las perplejidades en validación, finalmente pruebe una vez en test. Para esto puede explorar algunos valores $\lambda$ y elija el mejor, i.e., 
$$
\lambda_1 = \left[\frac{1}{3}, \frac{1}{3}, \frac{1}{3}\right] \\
\lambda_2 = [0.4, 0.4, 0.2] \\ 
\lambda_3 = [0.2, 0.4, 0.4] \\ 
\lambda_4 = [0.5, 0.4, 0.1] \\ 
\lambda_5 =[0.1, 0.4, 0.5] \\ \nonumber
$$

In [10]:
tweets.obtain_perplexity(use_data_test=True)

Lambda                                                          Perplexity
------------------------------------------------------------  ------------
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]       2.2506
[0.4, 0.4, 0.2]                                                    2.35088
[0.2, 0.4, 0.4]                                                    2.20592
[0.5, 0.4, 0.1]                                                    2.48767
[0.1, 0.4, 0.5]                                                    2.1582


In [11]:
tweets.obtain_perplexity(use_data_test=False)

Lambda                                                          Perplexity
------------------------------------------------------------  ------------
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]       2.15777
[0.4, 0.4, 0.2]                                                    2.25321
[0.2, 0.4, 0.4]                                                    2.11478
[0.5, 0.4, 0.1]                                                    2.38302
[0.1, 0.4, 0.5]                                                    2.06914


### 3) Generación de texto

#### Punto 1

Proponga una estrategia con base en Expectation Maximization para encontrar buenos valores de interpolación en $\hat{P}$ usando todo el dataset de agresividad. Para ello experimente con el modelo en particiones de 80%, 10% y 10% para entrenar (train), ajustar parámetros (val) y probar (test) respectivamente. 1 Muestre como bajan las perplejidades en 5 iteraciones que usted elija (de todas las que sean necesarias de acuerdo a su EM) en validación, y pruebe una vez en test. Sino logra hacer este punto, haga los siguientes dos con el modelo de lenguaje con $\lambda$ fijos.

In [12]:
lambdas = tweets.language_model.apply_expectation_maximization()

Iteracion    lambdas                                                               Suma    Perplexidad
-----------  ------------------------------------------------------------------  ------  -------------
Inicio       [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]             1        2.2506
Iteración 1  [1.0751024666888301e-09, 2.9347965000825594e-08, 0.99999999999994]       1        2.02979
Iteración 2  [1.2952031643886892e-36, 1.5460152938022499e-31, 1.0]                    1        2.02979
Iteración 3  [2.2646548008525604e-117, 2.260066097622663e-101, 1.0]                   1        2.02979
Iteración 4  [0.0, 7.0606259637826e-311, 1.0]                                         1        2.02979
Iteración 5  [0.0, 0.0, 1.0]                                                          1        2.02979


In [35]:
from Modules.models import tweetear_model
tweetear = tweetear_model(tweets.language_model, [1, 0, 0])

#### Punto 2

Haga una función "tuitear" con base en su modelo de lenguaje $\hat{P}$ del último punto. El modelo deberá poder parar automáticamente cuando genere el símbolo de terminación de tuit al final (e.g., ``</s>``), o 50 palabras. Proponga algo para que en los últimos tokens sea más probable generar el token ``</s>``. Muestre al menos cinco ejemplos.

In [14]:
tweetear.write("<s> me")

'<s> me caga que me <unk> </s>'

In [15]:
tweetear.write("<s> hola")

'<s> hola pinche putita te pones bien cachonda hija de tu puta madre </s>'

In [36]:
tweetear.write("<s> GRACIAS")

'<s> gracias facebook pero no son personas que <unk> </s>'

In [17]:
tweetear.write("<s> gracias por")

'<s> gracias por resolver mi duda existencial </s>'

In [18]:
tweetear.write("<s> pisando")

'<s> pisando <unk> </s>'

In [19]:
tweetear.write("<s> vale")

'<s> vale verga </s>'

#### Punto 3
Use la intuición que ha ganado en esta tarea y los datos de las mañaneras para entrenar un modelo de lenguaje AMLO. Haga una un función "dar_conferencia()". Generé un discurso de 300 palabras y detenga al modelo de forma abrupta

In [20]:
from Modules.mañaneras import AMLO_conferences_model
parameters["max words"]=20000
conferences = AMLO_conferences_model(parameters)

In [21]:
tweetear = tweetear_model(conferences.language_model, [1,0,0])

In [22]:
tweetear.write("presidente es", 20)

'presidente es que se <unk> a la gente que no se puede hacer una revision de los estados unidos y canada y'

In [23]:
tweetear.write("buenos dias hoy", 300)

'buenos dias hoy vamos a tener una reunion con el gobierno de mexico y en el caso de la conferencia de prensa matutina del presidente andres manuel lopez obrador si pero no es un asunto de los estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y estados unidos y canada y e

#### Punto 4
Calcule el estimado de cada uno sus modelos de lenguaje (el de tuits y el de amlo) para las frases: "sino gano me voy a la chingada", "ya se va a acabar la corrupción".

##### Modelo de tweets

In [24]:
tweets.language_model.tweet_probability("sino gano me voy a la chingada",[0,0,1])

2.1700611388168264e-16

In [25]:
tweets.language_model.tweet_probability("ya se va a acabar la corrupción",[0,0,1])

9.574399840811954e-15

##### Modelo con conferencias

In [26]:
conferences.language_model.tweet_probability("sino gano me voy a la chingada",[0,0,1])

3.1874064783191074e-20

In [27]:
conferences.language_model.tweet_probability("ya se va a acabar la corrupción",[0,0,1])

8.326693748476923e-17

#### Punto 5
Para cada oración del punto anterior, haga todas las permutaciones posibles. Calcule su probabilidad a cada nueva frase y muestre el top 3 mas probable y el top 3 menos probable (para ambos modelos de lenguaje). Proponga una frase más y haga lo mismo

In [33]:
from itertools import permutations
from tabulate import tabulate
dataset = {"pharses": ["sino gano me voy a la chingada",
                       "ya se va a acabar la corrupción",
                       "hoy quiero decir"],
           "models": {"Conferencia": conferences.language_model.tweet_probability,
                      "Tweets": tweets.language_model.tweet_probability}}
resultados = []
for pharse in dataset["pharses"]:
    print("\n{}".format("-"*40))
    tokens = tokenize(pharse)
    tokens_len = len(tokens)
    results = []
    for permutation in permutations(tokens, tokens_len):
        pharse_permutation = " ".join(permutation)
        for model in dataset["models"]:
            function = dataset["models"][model]
            probability = function(pharse_permutation, [0, 0, 1])
            results += [[model, pharse_permutation, probability]]
    results = sorted(results,
                     key=lambda x: x[2],
                     reverse=True)
    results = results[:5]
    print(tabulate(results,
                   headers=["Modelo",
                            "Palabra",
                            "Probabilidad"]))


----------------------------------------
Modelo    Palabra                           Probabilidad
--------  ------------------------------  --------------
Tweets    la a chingada me voy sino gano     4.52383e-13
Tweets    la a chingada me voy gano sino     4.52383e-13
Tweets    la voy me chingada a sino gano     4.52383e-13
Tweets    la voy me chingada a gano sino     4.52383e-13
Tweets    la voy chingada me a sino gano     4.52383e-13

----------------------------------------
Modelo    Palabra                            Probabilidad
--------  -------------------------------  --------------
Tweets    ya se la va a acabar corrupcion       2.317e-12
Tweets    ya se la va a corrupcion acabar       2.317e-12
Tweets    ya se la a va acabar corrupcion       2.317e-12
Tweets    ya se la a va corrupcion acabar       2.317e-12
Tweets    ya va a se la acabar corrupcion       2.317e-12

----------------------------------------
Modelo    Palabra             Probabilidad
--------  ----------------